<a href="https://colab.research.google.com/github/akshitagupta15june/OSASense/blob/main/osasense.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 4339071086288115981, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14674281152
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 4248203340825268888
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [ ]:
from tensorflow import keras
import sklearn
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Dense,Dropout,LSTM
from tensorflow.keras.models import Sequential
from google.colab import files

In [ ]:
df='sleeep_apnea_data.xlsx'
main_df=pd.read_excel(df,'male')


In [ ]:
df=main_df.sample(frac=1).reset_index(drop=True)
df.dropna(inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [ ]:
df.Gender=le.fit_transform(df.Gender)

In [ ]:
def fun(num): 
  
    if num<=30: 
        return 0
    else: 
        return 1

In [ ]:
df['status']=df.AHI.apply(fun)
status=["negative","positive"]


In [ ]:
df

,Gender,BQ,ESS,BMI,Weight,Height,Head,Neck,Waist,Buttock,Age,AHI,status
0,0,0.0,9.0,26.291724,69.0,162.0,53.6,36.0,87.5,98.0,58.0,30.000000,0
1,1,0.0,8.0,22.183919,69.5,177.0,56.0,37.0,85.0,96.0,45.0,29.020772,0
2,1,0.0,0.0,24.258674,76.0,177.0,55.5,36.4,83.5,96.0,38.0,34.859155,1
3,0,0.0,9.0,28.338234,73.0,160.5,55.0,34.3,86.0,104.0,36.0,1.660000,0
4,1,1.0,12.0,43.598890,132.0,174.0,62.0,51.5,133.0,129.0,35.0,75.380282,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5240,1,0.0,5.0,25.854639,81.0,177.0,56.0,38.0,89.0,100.5,39.0,52.784810,1
5241,1,0.0,19.0,26.609713,76.0,169.0,60.0,38.0,94.0,107.0,56.0,41.079545,1
5242,1,1.0,7.0,32.461911,75.0,152.0,56.0,41.0,94.5,105.0,43.0,62.107728,1
5243,1,0.0,13.0,30.778701,90.0,171.0,59.5,43.0,113.8,98.0,35.0,107.491166,1


In [ ]:
df1=df.drop(['Gender','status'],1)

In [ ]:
df1['Gender']=df['Gender']
df1["status"]=df['status']

In [ ]:
df1

,BQ,ESS,BMI,Weight,Height,Head,Neck,Waist,Buttock,Age,AHI,Gender,status
0,0.0,9.0,26.291724,69.0,162.0,53.6,36.0,87.5,98.0,58.0,30.000000,0,0
1,0.0,8.0,22.183919,69.5,177.0,56.0,37.0,85.0,96.0,45.0,29.020772,1,0
2,0.0,0.0,24.258674,76.0,177.0,55.5,36.4,83.5,96.0,38.0,34.859155,1,1
3,0.0,9.0,28.338234,73.0,160.5,55.0,34.3,86.0,104.0,36.0,1.660000,0,0
4,1.0,12.0,43.598890,132.0,174.0,62.0,51.5,133.0,129.0,35.0,75.380282,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5240,0.0,5.0,25.854639,81.0,177.0,56.0,38.0,89.0,100.5,39.0,52.784810,1,1
5241,0.0,19.0,26.609713,76.0,169.0,60.0,38.0,94.0,107.0,56.0,41.079545,1,1
5242,1.0,7.0,32.461911,75.0,152.0,56.0,41.0,94.5,105.0,43.0,62.107728,1,1
5243,0.0,13.0,30.778701,90.0,171.0,59.5,43.0,113.8,98.0,35.0,107.491166,1,1


In [ ]:
x=np.array(df1.drop(['AHI',"status"],1))
y=np.array(df1['status'])

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20)


In [ ]:
from sklearn import linear_model,svm, tree
from sklearn.linear_model import LogisticRegression,Ridge
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor

In [ ]:
model1= linear_model.LinearRegression(fit_intercept=True)
svr_poly = svm.SVR(kernel='linear', C=100, gamma='auto', degree=3, epsilon=.1,coef0=1)
dtree = DecisionTreeRegressor(max_depth=10, min_samples_leaf=0.13, random_state=3)
sv=svm.SVC(C=10,kernel='linear')

In [ ]:
sv.fit(x_train,y_train)

SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
acc=sv.score(x_test,y_test)

In [ ]:
acc

0.776930409914204

In [ ]:
p1=sv.predict(x_test)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
#from tensorflow.python.keras.layers.kernelized import RandomFourierFeatures
from tensorflow.keras import layers


In [ ]:
model=Sequential()
model.add(Dense(30,activation='relu',input_shape=[x_train.shape[1]]))
model.add(Dropout(.02))
model.add(Dense(10,activation='relu'))
model.add(Dropout(.02))
model.add(Dense(50,activation='relu'))
#model.add(Dense(50,activation='relu'))

model.add(Dense(1,activation='sigmoid'))
model.compile(Adam(lr=0.01),loss='binary_crossentropy',metrics=['accuracy'])
model.fit(x_train,y_train,epochs=100,validation_split=0.2)

Epoch 1/100
105/105 [==============================] - 4s 6ms/step - loss: 1.9295 - accuracy: 0.5237 - val_loss: 0.6595 - val_accuracy: 0.5864
Epoch 2/100
105/105 [==============================] - 0s 3ms/step - loss: 0.6795 - accuracy: 0.5520 - val_loss: 0.6825 - val_accuracy: 0.5864
Epoch 3/100
105/105 [==============================] - 0s 3ms/step - loss: 0.6843 - accuracy: 0.5675 - val_loss: 0.5839 - val_accuracy: 0.5948
Epoch 4/100
105/105 [==============================] - 0s 3ms/step - loss: 0.6167 - accuracy: 0.6042 - val_loss: 0.5890 - val_accuracy: 0.6937
Epoch 5/100
105/105 [==============================] - 0s 3ms/step - loss: 0.5984 - accuracy: 0.7081 - val_loss: 0.4968 - val_accuracy: 0.7878
Epoch 6/100
105/105 [==============================] - 0s 3ms/step - loss: 0.5240 - accuracy: 0.7455 - val_loss: 0.4637 - val_accuracy: 0.7878
Epoch 7/100
105/105 [==============================] - 0s 3ms/step - loss: 0.5546 - accuracy: 0.7209 - val_loss: 0.4626 - val_accuracy: 0.7878

In [ ]:
model.predict([[  0.  ,   0.  ,  24.22,  70.  , 170.  ,  58.5 ,  42.  ,  92.5 ,
        99.  ,  60.  ,   1. ]])

array([[0.42722225]], dtype=float32)

In [ ]:


p=model.predict(x_test)



In [ ]:
p[1047]

array([0.00998751], dtype=float32)

In [ ]:
y_test[1047]

0

In [ ]:
keras_file="model_tensor"
keras.models.save_model(model,keras_file)

INFO:tensorflow:Assets written to: model_tensor/assets


In [ ]:
import tensorflow
converter = tensorflow.lite.TFLiteConverter.from_saved_model(keras_file)
tflite_model=converter.convert()
open("tflitemodel.tflite","wb").write(tflite_model)


7024